https://github.com/ZhiqingXiao/rl-book/blob/master/chapter02_mdp/CliffWalking-v0.ipynb

In [1]:
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [2]:
import numpy as np
np.random.seed(0)
import scipy
import gym

In [3]:
env = gym.make('CliffWalking-v0')
env.seed(0)
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('状态数量 = {}, 动作数量 = {}'.format(env.nS, env.nA))
print('地图大小 = {}'.format(env.shape))

观测空间 = Discrete(48)
动作空间 = Discrete(4)
状态数量 = 48, 动作数量 = 4
地图大小 = (4, 12)


运行一回合

In [4]:
def play_once(env, policy):
    total_reward = 0
    state = env.reset()
    while True:
        loc = np.unravel_index(state, env.shape)
        print('状态 = {}, 位置 = {}'.format(state, loc), end=' ')
        action = np.random.choice(env.nA, p=policy[state])
        next_state, reward, done, _ = env.step(action)
        print('动作 = {}, 奖励 = {}'.format(action, reward))
        total_reward += reward
        if done:
            break
        state = next_state
    return total_reward

用最优策略运行一回合

In [5]:
actions = np.ones(env.shape, dtype=int)
actions[-1, :] = 0
actions[:, -1] = 2
optimal_policy = np.eye(4)[actions.reshape(-1)]

In [6]:
total_reward = play_once(env, optimal_policy)
print('回合奖励 = {}'.format(total_reward))

状态 = 36, 位置 = (3, 0) 动作 = 0, 奖励 = -1
状态 = 24, 位置 = (2, 0) 动作 = 1, 奖励 = -1
状态 = 25, 位置 = (2, 1) 动作 = 1, 奖励 = -1
状态 = 26, 位置 = (2, 2) 动作 = 1, 奖励 = -1
状态 = 27, 位置 = (2, 3) 动作 = 1, 奖励 = -1
状态 = 28, 位置 = (2, 4) 动作 = 1, 奖励 = -1
状态 = 29, 位置 = (2, 5) 动作 = 1, 奖励 = -1
状态 = 30, 位置 = (2, 6) 动作 = 1, 奖励 = -1
状态 = 31, 位置 = (2, 7) 动作 = 1, 奖励 = -1
状态 = 32, 位置 = (2, 8) 动作 = 1, 奖励 = -1
状态 = 33, 位置 = (2, 9) 动作 = 1, 奖励 = -1
状态 = 34, 位置 = (2, 10) 动作 = 1, 奖励 = -1
状态 = 35, 位置 = (2, 11) 动作 = 2, 奖励 = -1
回合奖励 = -13


## 求解 Bellman 期望方程

In [7]:
def evaluate_bellman(env, policy, gamma=1.):
    a, b = np.eye(env.nS), np.zeros((env.nS))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            pi = policy[state][action]
            for p, next_state, reward, done in env.P[state][action]:
                a[state, next_state] -= (pi * gamma * p)
                b[state] += (pi * reward * p)
    v = np.linalg.solve(a, b)
    q = np.zeros((env.nS, env.nA))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            for p, next_state, reward, done in env.P[state][action]:
                q[state][action] += ((reward + gamma * v[next_state]) * p)
    return v, q

In [8]:
policy = np.random.uniform(size=(env.nS, env.nA))
policy = policy / np.sum(policy, axis=1)[:, np.newaxis]

state_values, action_values = evaluate_bellman(env, policy)
print('状态价值 = {}'.format(state_values))
print('动作价值 = {}'.format(action_values))

状态价值 = [-71259.73993046 -71237.78971176 -71194.84815307 -71140.80283084
 -71061.70657407 -70938.9109303  -68108.48102393 -66319.10665149
 -64517.35282333 -61466.53107191 -57841.39233559 -49099.24383729
 -71264.96522919 -71247.53092065 -71217.52824196 -71120.56871128
 -71054.59876056 -70900.38265543 -69089.0248754  -67348.61705426
 -65077.28575314 -59299.02626942 -54477.04027155 -43408.92846114
 -71343.97588504 -71365.86170718 -71356.29488211 -71310.51896579
 -71248.19189085 -70916.43626142 -70432.29357881 -69519.4952358
 -69113.6008844  -66843.01499868 -60740.99702848 -33361.65231114
 -71417.8714599  -71463.16859933 -71447.66816433 -71480.47316827
 -71514.26925748 -71213.20010573 -71233.56329316 -71021.31099414
 -70840.15827719 -69680.13842414 -65008.3404254       0.        ]
动作价值 = [[-7.12607399e+04 -7.12387897e+04 -7.12659652e+04 -7.12607399e+04]
 [-7.12387897e+04 -7.11958482e+04 -7.12485309e+04 -7.12607399e+04]
 [-7.11958482e+04 -7.11418028e+04 -7.12185282e+04 -7.12387897e+04]
 [-7.

评估最优策略的价值

In [9]:
optimal_state_values, optimal_action_values = evaluate_bellman(env, optimal_policy)
print('最优状态价值 = {}'.format(optimal_state_values))
print('最优动作价值 = {}'.format(optimal_action_values))

最优状态价值 = [-14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3. -13. -12.
 -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2. -12. -11. -10.  -9.
  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1. -13. -12. -11. -10.  -9.  -8.
  -7.  -6.  -5.  -4.  -3.   0.]
最优动作价值 = [[ -15.  -14.  -14.  -15.]
 [ -14.  -13.  -13.  -15.]
 [ -13.  -12.  -12.  -14.]
 [ -12.  -11.  -11.  -13.]
 [ -11.  -10.  -10.  -12.]
 [ -10.   -9.   -9.  -11.]
 [  -9.   -8.   -8.  -10.]
 [  -8.   -7.   -7.   -9.]
 [  -7.   -6.   -6.   -8.]
 [  -6.   -5.   -5.   -7.]
 [  -5.   -4.   -4.   -6.]
 [  -4.   -4.   -3.   -5.]
 [ -15.  -13.  -13.  -14.]
 [ -14.  -12.  -12.  -14.]
 [ -13.  -11.  -11.  -13.]
 [ -12.  -10.  -10.  -12.]
 [ -11.   -9.   -9.  -11.]
 [ -10.   -8.   -8.  -10.]
 [  -9.   -7.   -7.   -9.]
 [  -8.   -6.   -6.   -8.]
 [  -7.   -5.   -5.   -7.]
 [  -6.   -4.   -4.   -6.]
 [  -5.   -3.   -3.   -5.]
 [  -4.   -3.   -2.   -4.]
 [ -14.  -12.  -14.  -13.]
 [ -13.  -11. -113.  -13.]
 [ -12.  -10. -113.  -12.]
 [ -11. 

In [ ]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0. # 记录回合总奖励，初始化为0
    observation = env.reset() # 重置游戏环境，开始新回合
    while True: # 不断循环，直到回合结束
        if render: # 判断是否显示
            env.render() # 显示图形界面，图形界面可以用 env.close() 语句关闭
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action) # 执行动作
        episode_reward += reward # 收集回合奖励
        if train: # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done: # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward # 返回回合总奖励

## 求解 Bellman 最优方程

In [10]:
def optimal_bellman(env, gamma=1.):
    p = np.zeros((env.nS, env.nA, env.nS))
    r = np.zeros((env.nS, env.nA))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            for prob, next_state, reward, done in env.P[state][action]:
                p[state, action, next_state] += prob
                r[state, action] += (reward * prob)
    c = np.ones(env.nS)
    a_ub = gamma * p.reshape(-1, env.nS) - \
            np.repeat(np.eye(env.nS), env.nA, axis=0)
    b_ub = -r.reshape(-1)
    a_eq = np.zeros((0, env.nS))
    b_eq = np.zeros(0)
    bounds = [(None, None),] * env.nS
    res = scipy.optimize.linprog(c, a_ub, b_ub, bounds=bounds,
            method='interior-point')
    v = res.x
    q = r + gamma * np.dot(p, v)
    return v, q

In [11]:
optimal_state_values, optimal_action_values = optimal_bellman(env)
print('最优状态价值 = {}'.format(optimal_state_values))
print('最优动作价值 = {}'.format(optimal_action_values))

最优状态价值 = [-1.40000000e+01 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01
 -1.00000000e+01 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00
 -6.00000000e+00 -5.00000000e+00 -4.00000000e+00 -3.00000000e+00
 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01
 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00
 -5.00000000e+00 -4.00000000e+00 -3.00000000e+00 -2.00000000e+00
 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01 -9.00000000e+00
 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00 -5.00000000e+00
 -4.00000000e+00 -3.00000000e+00 -2.00000000e+00 -1.00000000e+00
 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01
 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00
 -5.00000000e+00 -4.00000000e+00 -9.99999999e-01  1.82288980e-11]
最优动作价值 = [[ -14.99999999  -13.99999999  -13.99999999  -14.99999999]
 [ -13.99999999  -13.          -13.          -14.99999999]
 [ -13.          -12.          -12.          -13.99999999]
 [ -12.          -11.   

In [12]:
optimal_actions = optimal_action_values.argmax(axis=1)
print('最优策略 = {}'.format(optimal_actions))

最优策略 = [2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 0
 0 0 0 0 0 0 0 0 0 1 0]
